## Тестовое задание для SLabAI
### Выполнил Эдуард Аюнц, eayunts@gmail.com

На странице FrozenLake на OpenAI можно найти приведенный ниже код, в нем используется алгоритм Q Learning, использующий уравнение Беллмана для динамической оптимизации. Данное решение дает точность порядка 50-60%,  и при написании персептрона, я ориентировался именно на это алгоритм, с той лишь разницой, что вместо уравнения Беллмана  сеть будет сама подбирать оптимальные веса, которые для данной инициализации ожидаемых выигрышей при каждом состоянии для каждого действия выведут действительный ожидаемый выигрыш

In [1194]:
import numpy as np
import gym
env = gym.make('FrozenLake-v0')

[2017-01-25 19:20:23,288] Making new env: FrozenLake-v0


In [1195]:
Q = np.zeros([env.observation_space.n,env.action_space.n])
lr = .7
y = .99
num_episodes = 1000
rList = []
for i in range(num_episodes):
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    while j < 99:
        j+=1
        L = Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1))
        a = np.argmax(L)
        s1,r,d,_ = env.step(a)
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a]) #уравнение Беллмана, то же самое ожидается от персептрона.
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)
print("Score over time: " +  str(sum(rList)/(num_episodes)))

Score over time: 0.401


Теперь непосредственно моя реализация.

In [1627]:
class NeuralNetwork:    
    
    def __init__(self, num_features, hidden_size, seed=None):
        env = gym.make('FrozenLake-v0')
        self.num_features = num_features
        self.hidden_size = hidden_size
        self.init_weights(seed)
    
    def init_weights(self, seed):
        np.random.seed(seed)
        self.W1 = np.abs(np.random.normal(size=(self.num_features, self.hidden_size), scale=0.1) )
        self.b1 = np.abs(np.random.normal(size=self.hidden_size))
        self.W2 = np.abs(np.random.normal(size=(self.hidden_size, 1), scale=0.1))
        self.b2 = np.abs(np.random.normal(size=64)) # на выход подается массив из 64 элементов
        #идея в том, чтоб подавать на вход и получать матрицу ожидаемых выигрышей, которая состоин из 16*4=64 элементов
        #нумерация следующая - первые 4 элемента - ожидаемые выигрыше при каждом действии в начальной точке, следующая четверка - 
        #из состояние с индексом 1 и так далее.
        #следовательно,на выходе должны получаться та же таблица но с верными ожидаемыми выигрышами.

        
    def relu(self, x): # в качестве функции активации будем использовать  ReLU
        return np.maximum(x, 0)
    
    def relu_grad(self, x):
        return np.where(x > 0, 1, 0)
    
    def loss_gradient(self, X, y, s):
        hidden = np.dot(X, self.W1) + self.b1 #персептрон
        activation = self.relu(hidden)
        answer = np.dot(activation, self.W2) + self.b2
        s,r,d,_= env.step(np.argmax(answer[s*4:s*4+4]+np.random.rand(4))) #добавляю случайный вектор чтоб обогатить сеть
       
        global p # число сыгранных игр
        global q # число побед
        global m # число шагов в каждой игре
        global error #производная ошибки
        global n_steps #массив с числом шагов
        
        if d == True and r ==0: #если проигрыш, то ошибка равна 1
            s = env.reset()
            error = 1
            n_steps.append(m)
            p+=1 #увеличиваем число игр
            m=1
        elif d == True and r ==1: #при выигрыше ошибка равняется 0
            s = env.reset()
            error = 0
            q+=1 #увеличиваем число побед
            n_steps.append(m) # записываем в список число шагов до конца игры, для статистики
            m=1
        else: 
            m+=1 #число проделанных шагов
            error = 1-m/20 #если игра продолжается, то ошибка уменьшается с ростом числа проделанных шагов
        
        db2 = np.sum(error) #обратное распространение ошибки
        dW2 = activation.T.dot(error)

        dhidden = self.relu_grad(hidden) * error * self.W2.T
        db1 = dhidden.mean(0)
        dW1 = np.array([X.T]).T.dot(dhidden)

        return dW1, db1, dW2, db2

    def gradient_descent(self, X, y, learning_rate,s):
        dW1, db1, dW2, db2 = self.loss_gradient(X, y,s) 
        self.W1 -= dW1 * learning_rate #обновляем параметры
        self.b1 -= db1 * learning_rate
        dW2=dW2 * learning_rate
        for i in range(100): #100 так как сто нейроной в скрытом слое
            self.W2[i][0]-=dW2[i]
        self.b2 -= db2 * learning_rate
        
    def fit(self, X, y, epochs, learning_rate):
        s = env.reset()
        s,r,d,_ = env.step(int(np.random.choice(4, 1))) #первый шаг делаем случайно
        for epoch in range(epochs):
            self.gradient_descent(X, y, learning_rate/(epoch+1),s) #добавил learning decay, при каждой эпохе темп 
             #изменения параметром будет снижаться

В тестах экспериментировал с числом скрытых слоев, однако наилучший результат получался при числе порядка 100.

In [1644]:
X = np.abs(np.random.uniform(size =(64))) # инициализиурем случайно распределение матрицы ожидаемых выигрышей.

In [1645]:
nn = NeuralNetwork(64, hidden_size=100, seed=41)

[2017-01-25 20:47:30,074] Making new env: FrozenLake-v0


### Тесты

С learning rate = 0.1

In [1647]:
for i in range(10):
    p=0  #счетчики
    q=0
    m=1
    n_steps=[]
    error=1
    nn.fit(X, 1, epochs=1000, learning_rate=0.1)
    print(p, 'games played \n', q, 'victories \n', np.round(100*q/p,2), " percent accuracy" )
    print(np.round(np.mean(n_steps),1), 'mean n of steps \n')

68 games played 
 0 victories 
 0.0  percent accuracy
14.4 mean n of steps 

176 games played 
 17 victories 
 9.66  percent accuracy
5.2 mean n of steps 

172 games played 
 14 victories 
 8.14  percent accuracy
5.4 mean n of steps 

181 games played 
 9 victories 
 4.97  percent accuracy
5.2 mean n of steps 

176 games played 
 7 victories 
 3.98  percent accuracy
5.5 mean n of steps 

168 games played 
 10 victories 
 5.95  percent accuracy
5.6 mean n of steps 

165 games played 
 11 victories 
 6.67  percent accuracy
5.7 mean n of steps 

90 games played 
 0 victories 
 0.0  percent accuracy
11.0 mean n of steps 

57 games played 
 0 victories 
 0.0  percent accuracy
17.2 mean n of steps 

181 games played 
 12 victories 
 6.63  percent accuracy
5.2 mean n of steps 



Теперь попробуем learning rate = 0.001

In [1648]:
for i in range(10):
    p=0  #счетчики
    q=0
    m=1
    n_steps=[]
    error=1
    nn.fit(X, 1, epochs=1000, learning_rate=0.001)
    print(p, 'games played \n', q, 'victories \n', np.round(100*q/p,2), " percent accuracy" )
    print(np.round(np.mean(n_steps),1), 'mean n of steps \n')

190 games played 
 10 victories 
 5.26  percent accuracy
5.0 mean n of steps 

69 games played 
 0 victories 
 0.0  percent accuracy
14.1 mean n of steps 

183 games played 
 10 victories 
 5.46  percent accuracy
5.2 mean n of steps 

78 games played 
 0 victories 
 0.0  percent accuracy
12.8 mean n of steps 

183 games played 
 8 victories 
 4.37  percent accuracy
5.2 mean n of steps 

60 games played 
 0 victories 
 0.0  percent accuracy
16.0 mean n of steps 

172 games played 
 10 victories 
 5.81  percent accuracy
5.5 mean n of steps 

184 games played 
 10 victories 
 5.43  percent accuracy
5.1 mean n of steps 

188 games played 
 6 victories 
 3.19  percent accuracy
5.1 mean n of steps 

167 games played 
 9 victories 
 5.39  percent accuracy
5.6 mean n of steps 



Из интересных наблюдений - чередование нулевых запусков итераций с теми, которы набирают 5-6% выигрышей

In [1654]:
y=[]

In [1679]:
for i in range(100):
    p=0  #счетчики
    q=0
    m=1
    n_steps=[]
    error=1
    nn.fit(X, 1, epochs=1000, learning_rate=0.001)
    #print(p, 'games played \n', q, 'victories \n', np.round(100*q/p,2), " percent accuracy" )
    #print(np.round(np.mean(n_steps),1), 'mean n of steps \n')
    y.append(np.round(100*q/p,2))
print('Средняя точность:',np.mean(y))
print('Максимальная точность',np.max(y))
x=[]
for i in range(len(y)):
    if y[i]>1: x.append(y[i])
print('Средняя точность среди успешных запусков', np.mean(x))
print('Успешными посчитал запуски с точностью выше 1%')

Средняя точность: 2.08256179775
Максимальная точность 10.9
Средняя точность среди успешных запусков 4.80140625
Успешными посчитал запуски с точностью выше 1%


In [1408]:
np.max(x) #максимальная достигнутая точность в ходе все тестов - 12% 

12.0

В целом это есть достигнутый мной результат, максимальная точность - 12%,  к сожалению различные эксперименты не позволили мне повысить качество алгоритма, 
буду рад услышать фидбек и узнать допущенные ошибки.